In [1]:
from pathlib import Path
import re
import numpy as np
import pandas as pd
from typing import List, Optional
import io
import requests
import zipfile

# Create data directory
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

# Direct Kaggle dataset URLs (these are public)
URLS = {
    "Netflix": "https://www.kaggle.com/api/v1/datasets/download/shivamb/netflix-shows",
    "Amazon Prime": "https://www.kaggle.com/api/v1/datasets/download/shivamb/amazon-prime-movies-and-tv-shows",
    "Disney+": "https://www.kaggle.com/api/v1/datasets/download/shivamb/disney-movies-and-tv-shows"
}

def download_and_extract(url, dest_folder):
    print(f"Downloading from {url} ...")
    response = requests.get(url, stream=True)
    if response.status_code != 200:
        raise Exception(f"Failed to download {url} – Kaggle may require authentication.")
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(dest_folder)
    print(f"Extracted to {dest_folder}")

for name, url in URLS.items():
    folder = DATA_DIR / name.replace(" ", "_").lower()
    folder.mkdir(exist_ok=True)
    download_and_extract(url, folder)

# Identify the CSVs
FILES = {
    "Netflix": next((DATA_DIR / "netflix").rglob("*.csv")),
    "Amazon Prime": next((DATA_DIR / "amazon_prime").rglob("*.csv")),
    "Disney+": next((DATA_DIR / "disney+").rglob("*.csv")),
}

for k, v in FILES.items():
    print(k, "->", v)
print("Done")

Extracted to data/netflix
Extracted to data/amazon_prime
Extracted to data/disney+
Netflix -> data/netflix/netflix_titles.csv
Amazon Prime -> data/amazon_prime/amazon_prime_titles.csv
Disney+ -> data/disney+/disney_plus_titles.csv
Done


In [2]:
def load_any_df(path: Path) -> pd.DataFrame:
    ext = path.suffix.lower()
    if ext == ".csv":
        return pd.read_csv(path)
    # JSON: try standard then JSON-lines
    try:
        return pd.read_json(path, lines=False)
    except ValueError:
        return pd.read_json(path, lines=True)

def to_snake(name: str) -> str:
    name = re.sub(r"[^\w]+", "_", name.strip())
    name = re.sub(r"([a-z0-9])([A-Z])", r"\1_\2", name)
    return name.lower().strip("_")

def get_col(df: pd.DataFrame, aliases: List[str]) -> Optional[str]:
    # Try snake-case alias match
    snake = {to_snake(c): c for c in df.columns}
    for a in aliases:
        if a in snake:
            return snake[a]
    # Fallback: case-insensitive plain match
    lowers = [al.replace("_", " ") for al in aliases]
    for c in df.columns:
        if c.lower() in lowers:
            return c
    return None

STANDARD_COLS = {
    "title": ["title", "show_title", "name"],
    "type": ["type", "content_type", "show_type"],
    "country": ["country", "countries", "country_of_origin"],
    "release_year": ["release_year", "year", "release_years"],
    "genres": ["genres", "genre", "listed_in", "categories"],
    "imdb_rating": ["imdb_rating", "imdb_score", "rating", "score"],
    "imdb_votes": ["imdb_votes", "votes", "imdb_vote_count"],
    "director": ["director", "directors"],
    "cast": ["cast", "actors", "starring"],
    "date_added": ["date_added", "added_on", "added"],
    "duration": ["duration", "runtime", "length"]
}

def coerce_schema(df: pd.DataFrame, platform: str) -> pd.DataFrame:
    out = pd.DataFrame()
    for std, aliases in STANDARD_COLS.items():
        aliases_snake = [to_snake(a) for a in aliases]
        if std not in aliases_snake:
            aliases_snake.append(std)
        col = get_col(df, aliases_snake)
        out[std] = df[col] if col is not None else np.nan
    out["platform"] = platform    
    return out



In [3]:
dfs = []
for plat, path in FILES.items():
    raw = load_any_df(path)
    std = coerce_schema(raw, plat)
    dfs.append(std)

print("Standardized shapes:")
for d in dfs:
    print(f"{d['platform'].iloc[0]} -> {d.shape}")

Standardized shapes:
Netflix -> (8807, 12)
Amazon Prime -> (9668, 12)
Disney+ -> (1450, 12)


In [4]:
def normalize_strings(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    for c in cols:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip()
            df.loc[df[c].isin(["nan", "NaN", "None"]), c] = np.nan
    return df

def to_numeric(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def safe_split(x, seps=(",", "|", ";")):
    if pd.isna(x): return []
    if isinstance(x, list): return [str(i).strip() for i in x if str(i).strip()]
    s = str(x)
    for sep in seps:
        s = s.replace(sep, ",")
    return [i.strip() for i in s.split(",") if i.strip()]

def split_multivalue_cols(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    for c in cols:
        if c in df.columns:
            df[c] = df[c].apply(safe_split)
    return df

cleaned = []
for df in dfs:
    df = df.copy()
    df = normalize_strings(df, ["title","type","country","genres","director","cast","date_added","duration"])
    df = to_numeric(df, ["release_year","imdb_rating","imdb_votes"])
    df = split_multivalue_cols(df, ["country","genres","cast","director"])
    cleaned.append(df)

print("Post-clean dtypes:")
cleaned[0].dtypes.head(12)
print (cleaned)


Post-clean dtypes:
[                      title     type          country  release_year  \
0      Dick Johnson Is Dead    Movie  [United States]          2020   
1             Blood & Water  TV Show   [South Africa]          2021   
2                 Ganglands  TV Show               []          2021   
3     Jailbirds New Orleans  TV Show               []          2021   
4              Kota Factory  TV Show          [India]          2021   
...                     ...      ...              ...           ...   
8802                 Zodiac    Movie  [United States]          2007   
8803            Zombie Dumb  TV Show               []          2018   
8804             Zombieland    Movie  [United States]          2009   
8805                   Zoom    Movie  [United States]          2006   
8806                 Zubaan    Movie          [India]          2015   

                                                 genres  imdb_rating  \
0                                       [Documentaries]

In [5]:
def _safe_median(s: pd.Series):
    s2 = s.dropna()
    return s2.median() if not s2.empty else np.nan

def impute_values_safely(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

# Fill numerical values with medians if it exists
    for c in ["imdb_rating", "imdb_votes", "release_year"]:
        if c in df.columns:
            med = _safe_median(df[c])
            if not np.isnan(med):
                df[c] = df[c].fillna(med)

# List categoricals ensuring non-empty lists
    def ensure_list(val):
        if isinstance(val, list):
            return val if len(val) else ["Unknown"]
        if pd.isna(val):
            return ["Unknown"]
        return [str(val)]

    for c in ["country","genres","cast","director"]:
        if c in df.columns:
            df[c] = df[c].apply(ensure_list)

# Fill Scalar strings only when missing
    for c in ["title","type","date_added","duration"]:
        if c in df.columns:
            df[c] = df[c].where(df[c].notna(), "Unknown")

    return df

imputed = [impute_values_safely(df) for df in cleaned]


In [6]:
def _normalize_title_for_dupes(s: pd.Series) -> pd.Series:
    def norm(x: str) -> str:
        if pd.isna(x): return x
        x = str(x).strip().lower()
        x = re.sub(r"\s+", " ", x)
        x = re.sub(r"[^\w\s]", "", x)
        return x
    return s.astype(str).apply(norm)

def drop_dupes_robust(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["__norm_title__"] = _normalize_title_for_dupes(df["title"]) if "title" in df.columns else np.nan
    keys = ["__norm_title__"]
    for k in ["platform","type","release_year"]:
        if k in df.columns:
            keys.append(k)
    before = len(df)
    df = df.drop_duplicates(subset=keys).reset_index(drop=True)
    after = len(df)
    print(f"De-duplicated: {before} → {after} (removed {before - after})")
    return df.drop(columns=["__norm_title__"])

deduped = [drop_dupes_robust(df) for df in imputed]


De-duplicated: 8807 → 8802 (removed 5)
De-duplicated: 9668 → 9659 (removed 9)
De-duplicated: 1450 → 1450 (removed 0)


In [7]:
combined_df = pd.concat(deduped, ignore_index=True)

print("Combined shape:", combined_df.shape)
print("\nCounts by platform:")
print(combined_df["platform"].value_counts(dropna=False))

if "type" in combined_df.columns:
    print("\nCounts by type:")
    print(combined_df["type"].value_counts(dropna=False))

missing = (combined_df.isna().mean().sort_values(ascending=False) * 100).round(1)
print("\nTop missingness (%):")
print(missing.head(12))

tmp = combined_df.copy()
tmp["__norm_title__"] = _normalize_title_for_dupes(tmp["title"])
keys = [k for k in ["__norm_title__","platform","type","release_year"] if k in tmp.columns]
dups = tmp.duplicated(subset=keys).sum()
print("\nRemaining duplicates by robust key:", dups)


Combined shape: (19911, 12)

Counts by platform:
platform
Amazon Prime    9659
Netflix         8802
Disney+         1450
Name: count, dtype: int64

Counts by type:
type
Movie      14984
TV Show     4927
Name: count, dtype: int64

Top missingness (%):
imdb_votes      100.0
imdb_rating      51.5
title             0.0
type              0.0
release_year      0.0
country           0.0
genres            0.0
director          0.0
cast              0.0
date_added        0.0
duration          0.0
platform          0.0
dtype: float64

Remaining duplicates by robust key: 0


In [8]:
combined_df.to_csv("clean_streaming_metadata.csv", index=False)

In [9]:
# Stats
df = pd.read_csv("outputs/clean_streaming_metadata.csv")



# General overview
print("Rows:", len(df))
print("Columns:", df.columns.tolist())

# Release year
print("\nRelease Year Stats:")
print(df['release_year'].describe())

# IMDb rating
print("\nIMDb Rating Stats:")
print(df['imdb_rating'].describe())

# Content type ratio
print("\nType distribution:")
print(df['type'].value_counts(normalize=True) * 100)

# Missing value percentages
print("\nMissing values (%):")
print(df.isnull().mean() * 100)



Rows: 19911
Columns: ['title', 'type', 'country', 'release_year', 'genres', 'imdb_rating', 'imdb_votes', 'director', 'cast', 'date_added', 'duration', 'platform']

Release Year Stats:
count    19911.000000
mean      2010.554116
std         15.949918
min       1920.000000
25%       2010.000000
50%       2016.000000
75%       2019.000000
max       2021.000000
Name: release_year, dtype: float64

IMDb Rating Stats:
count    9659.0
mean       16.0
std         0.0
min        16.0
25%        16.0
50%        16.0
75%        16.0
max        16.0
Name: imdb_rating, dtype: float64

Type distribution:
type
Movie      75.254884
TV Show    24.745116
Name: proportion, dtype: float64

Missing values (%):
title             0.000000
type              0.000000
country           0.000000
release_year      0.000000
genres            0.000000
imdb_rating      51.489127
imdb_votes      100.000000
director          0.000000
cast              0.000000
date_added        0.000000
duration          0.000000
platf